## 종만북 chapter 20. 문자열
문자열 검색 문제를 위한 **KMP 알고리즘**, 문자열 처리에 좋은 자료구로 **접미사 배열(suffix array)**를 소개.

#### 용어, 표기 정의
- 문자열 S의 길이는 |S|로 표기.  
- S의 i(0<=i<|S|)번 글자를 S[i]로 표기.
- 문자열 S의 i번 글자부터 j번 글자까지로 구성된 문자열을 S의 **부분 문자열(substring)**이라고 부르고, S[i...j]로 표기.
- 문자열 S의 0번 글자부터 a번 글자까지로 구성된 부분 문자열 S[0...a]를 S의 **접두사(prefix)**라고 부르며, S[0...a]로 표기.
- 문자열 S의 b번 글자부터 끝까지로 구성된 부분 문자열 S[b...|S|-1]을 S의 **접미사(suffix)**라고 부르며, S[b...|S|-1]로 표기.

### 문자열 검색
주어진 긴 문자열 H가 짧은 문자열 N을 부분 문자열로 포함하는지 확인하고, 포함한다면 N과 일치하는 부분 문자열의 시작 위치를 찾는 것.  
H = "hogwarts", N = "gwart"라 하면, H[2...6]=N이므로 H는 N을 포함하며, N의 시작 위치는 2가 된다.  
만약 N이 H안에서 두 번 이상 출현한다면 문자열 검색 알고리즘은 N이 출현하는 모든 위치를 반환해야 한다.  
예를 들어 H = "avava"는 N = "ava"를 0, 2 에서 포함.

이 문제를 푸는 가장 간단한 방법은 N의 가능한 모든 시작 위치를 다 시도해 보는 것.  
처음에는 0번 글자에서 시작하는 부분 문자열이 N과 같은지 확인하기 위해 H와 N의 글자를 하나하나 맞춰나가고,  
모든 글자가 일치한다면 그 위치를 답에 추가. 중간에 실패하거나 답을 찾은 후엔 시작 위치를 오른쪽 한 칸 이동하여 반복.  
    
    // 단순한 문자열 검색 알고리즘 구현
    vector<int> naiveSearch(const string& H, const string& N) {
        vector<int> ret;
        // 모든 시작 위치를 다 시도해본다.
        for (int begin = 0; begin + N.size() <= H.size(); begin++) {
            bool matched = true;
            for (int i = 0; i < N.size(); i++)
                if (H[begin + i] != N[i]) {
                    matched = false;
                    break;
                }
            if (matched) ret.push_back(begin);
        }
        return ret;
    }    

In [2]:
# 파이썬 구현
def naiveSearch(H, N):
    ret = []
    for begin in range(len(H) - len(N) + 1):
        matched = True
        for i in range(len(N)):
            if H[begin + i] != N[i]:
                matched = False
                break
        if matched:
            ret.append(begin)
    return ret

H = "avava"
N = "ava"
print(naiveSearch(H, N))

[0, 2]


특정 형태의 입력에 대해서는 엄청나게 비효율적으로 동작한다.  
H, N이 모두 a로만 구성된 긴 문자열이라고 하면 모든 시작 위치가 답이 된다.  
하지만 단순한 알고리즘은 모든 시작 위치에 대해서 비교를 수행하기 때문에 시간 복잡도는 O(|N|X|H|)가 된다.  
입력이 큰 경우 비효율적이지만 사실 이런 경우는 흔치 않고 구현이 매우 간단하기 때문에 표준 라이브러리 구현에 사용됨.  
C의 strstr(), C++ 문자열의 string::find(), 자바 문자열의 indexOf() 등..  

### KMP 알고리즘(Knuth-Morris-Pratt)
검색 과정에서 얻는 정보를 버리지 않고 잘 활용하면 많은 시간을 절약할 수 있다.  
어떤 긴 문자열에서 N = "aabaabac"를 찾는 경우에 시작 위치 i에서부터 맞춰가다 여덟번째 글자에서 불일치가 발생했다고 하면,  
단순 알고리즘은 i의 바로 다음위치인 i+1번 위치에서 다시 답을 찾기 시작한다.  
  
하지만 해당 부분 문자열 H[i...i+6]이 "aabaaba"임을 알기 때문에 i + 1에서 시작하는 H의 부분 문자열은 N과 일치할 수 없다.  
N[1] = a인데 H의 대응하는 글자가 b임을 아니까.  
이처럼 '현재 시작 위치에서 H와 N을 비교했을 때 몇 글자나 일치했는가'의 정보만 가지고 시작 위치 후보들을 걸러낼 수 있다.  

#### 다음 시작 위치 찾기
KMP알고리즘은 불일치가 일어났을 때 지금까지 일치한 글자의 수를 이용해 다음으로 시도해야 할 시작 위치를 빠르게 찾아준다.  
  
                         <---N[..matched-1]--->              
    N, begin = i         ooooooooo[-----A-----]x
    H                  __ooooooooo[-----B-----]________     
    N, begin = i + k              [-----C-----]--x


만약 위치 i에서 H와 N을 맞춰봤을 때 matched 만큼의 글자가 일치하고 다음 글자가 불일치했다면,  
matched만큼의 글자가 일치했기 때문에, N의 접두사 N[...matched-1]가 H[i...i+matched-1]와 일치한 것이다.  
이 때 만약 시작위치 i+k가 답이 될 수 있으려면 B와 C가 같아야 하고, 즉 A와 C가 같아야 한다.  
이때 A=C는 N[...matched-1]의 접두사이기도 하고 접미사이기도 하다.  
  
정리하면, 시작 위치 i+k에서 답을 찾을 수 있기 위해서는 N[...matched-1]의 길이 matched-k인 접두사와 접미사가 같아야 한다.  
그러므로 답이 될 수 있는 바로 다음 위치를 찾기 위해서는 N의 각 접두사에 대해 접두사도 되고 접미사도 되는 문자열의 최대 길이를 계산해 두면 된다.  
N = "aabaabac"에서 "aabaaba"의 접두사도 되고 접미사도 되는 문자열은 "aaba"와 "a"가 있지만 가장 긴 "aaba"를 이용해서 시작위치를 3만큼 옮겼다.  
KMP알고리즘은 전처리 과정에서 다음과 같이 정의되는 배열 pi[]를 계산한다.  
pi[i] = N[...i]의 접두사도 되고 접미사도 되는 문자열의 최대 길이.  
  
pi[]는 N이 어디까지 일치했는지가 주어질 때 다음 시작 위치를 어디로 해야할지 말해 주기 때문에,  
이를 **부분 일치 테이블(Partial Match Table)**이라고 부른다.  
  
N = "aabaabac"에 대해 각 접두사에 대해 KMP알고리즘이 미리 계산해 두는 부분 일치 테이블들의 값을 살펴보면,  

    i     N[...i]        접두사면서 접미사인 최대 문자열   pi[i]
    0       a                       None                      0
    1       aa                         a                      1
    2       aab                     None                      0   
    3       aaba                       a                      1
    4       aabaa                     aa                      2
    5       aabaab                   aab                      3
    6       aabaaba                 aaba                      4
    7       aabaabac                None                      0

#### 구현
KMP알고리즘은 시작위치 0에서부터 H와 N을 비교. matchted 글자가 일치한 후 불일치가 발생했다면 처음 그림에서 A의 길이는 pi[matched-1].  
따라서 시작 위치를 matched - pi[matched - 1]만큼 증가시키면 된다.  
또한 시작위치를 움직인 이후에도 N의 첫 pi[matched-1]글자는 대응되는 H의 글자와 일치한다는 걸 알고 있기 때문에,  
matched를 pi[matched-1]로 변경하고 비교를 계속하면 된다.  
답을 찾은 경우에는 현재 시작 위치를 답의 목록에 추가하는 것 외에는 불일치가 발생한 경우와 똑같다.  
matched=0인 경우 예외처리. (바로 다음시작위치에서 처음부터 검색 재개)

    vector<int> kmpSearch(const string& H, const string& N) {
        int n = H.size(), m = N.size();
        vector<int> ret;
        // pi[i] = N[..i]의 접미사도 되고 접두사도 되는 문자열의 최대 길이
        vector<int> pi = getPartialMatch(N);
        // begin = matched = 0부터 시작
        int begin = 0, matched = 0;
        while (begin <= n - m) {
            if (matched < m && H[begin + matched] == N[matched]) {
                ++matched;
                // m글자가 모두 일치했다면 답에 추가
                if (matched == m) ret.push_back(begin);
            }
            else {
                // 예외: matched가 0인 경우엔 다음 칸부터 계속
                if (matched == 0) ++begin;
                else {
                    begin += matched - pi[matched - 1];
                    // begin을 옮겼다고 처음부터 다시 비교할 필요 없다.
                    // 옮긴 후에도 pi[matched - 1]만큼은 항상 일치.
                    matchd = pi[matched - 1];
                }
            }
        }
        return ret;
    }
    
일단 getPartialMatch()를 제외하고 생각.  
while문에서 begin + matched는 절대 감소하지 않는다.  
따라서 한번 matched가 증가하고 나면, H[begin+matched]를 다시 참조할 일은 없다.  
문자 비교 성공은 각 문자당 최대 한 번씩만 일어나고 결과적으로 첫번째 if문은 최대 O(|H|)번 수행.  
또 문자 비교 실패 역시 최대 O(|H|)번 수행. 따라서 kmpSearch()의 반복문 전체 수행 횟수는 O(|H|)로 N에 관계없이 H에만 비례.

#### 부분 일치 테이블 생성.
KMP 검색 과정을 응용해 이 함수를 빠르게 구현할 수 있다.  
일단 가장 간단한 방법은 N의 각 접두사에 대해 가능한 답을 하나씩 모두 시도하는 것.  
길이 p인 접두사 N[...p-1]이 주어졌을 때 길이 p-1인 접두사, 길이 p-2인 접두사,...들을 확인하면서 이들이 N[...p-1]의 접미사가 되는지 확인.  
이 과정을 그대로 구현하면 각 접두사 길이의 제곱에 비례하는 시간이 걸리기 때문에, |N|개의 모든 접두사에 대해 수행하려면 $O(|N|^3)$.  
대개 |N|이 작기 때문에 이정도로 충분한 경우도 많지만 좀더 최적화할 수 있다.  
각 접두사에 대해 pi값을 따로 계산하는 것이 아니라 모든 접두사에 대해 한꺼번에 계산하는 것.  
두 글자 N[i]와 N[begin + i]가 일치할 때마다 pi[begin+i]를 갱신해주면 단순 문자열 검색 시간인 $O(|N|^2)$만에 부분 일치 테이블 계산.  
현재보다 왼쪽에 있는 시작 위치에서 이 위치의 값을 이미 갱신했을지도 모르기 때문에 max 사용..  
  
    // N에서 자기 자신을 찾으면서 나타나는 부분 일치를 이용해 pi[]를 계산.
    vector<int> getPartialMatchNaive(const string& N) {
        int m = N.size();
        vector<int> pi(m, 0);
        // 단순 문자열 검색 알고리즘 구현.
        for (int begin = 1; begin < m; begin++) {
            for (int i = 0; i + begin < m; i++) {
                if (N[begin + i] != N[i]) break;
                // i + 1 글자가 서로 대응.
                pi[begin + i] = max(pi[begin + i], i + 1);
            }
        }
        return pi;
    }

이런 검색과정을 다시 KMP로 구현할 수 있다.  
    
    vector<int> getPartialMatch(const string& N) {
        int m = N.size();
        vector<int> pi(m, 0);
        // KMP로 찾기.
        // N을 N에서 찾는다. begin = 0이면 자기 자신을 찾아버리니까 안됨.
        int begin = 1, matched = 0;
        // 비교할 문자가 N의 끝에 도달할 때까지 찾으면서 부분 일치를 모두 기록.
        while (begin + matched < m) {
            if (N[begin + matched] == N[matched]) {
                ++matched;
                pi[begin + matched - 1] = matched;
            }
            else {
                if (matched == 0)
                    ++begin;
                else {
                    begin += matched - pi[matched - 1];
                    matched = pi[matched - 1];
                }
            }
        }
        return pi;
    }
    
이 전처리 과정도 O(|N|). 모든 과정을 합하면 전체 O(|N|+|H|)가 됨.           

pi 계산의 문제. 혹은 $sp_i$  
    
    Text T와 Pattern P가 있을 때..
       1  2  3  4  5  6  7  8  9  10 11 12 13 14 15
    T  a  b  c  a  b  d  a  b  c   a  b  c  a  b  d ...
    P  a  b  c  a  b  d  a  b  c   a  b  d
    
위와 같은 경우 pi[11] = 5. 그러나 (11 - 5) =  6만큼 옮기고 나면 다시 바로 다음에서 불일치 발생..  
mismatch가 처음 난 값과 이동 후 mismatch된 값이 같게 되어 불필요한 비교가 한 번 더 발생.  
  
**$sp'_i$**를 새로 정의  
$sp_i$와 기본적으로 동일한데, Pattern[i+1]과 P[$sp'_i$+1]의 문자는 달라야 함!!

### 접두사/접미사 문제
#### 문제
아주대에 사는 외수는 작명에 능하기로 유명해서 많은 부부들이 아주대로 몰려와서 태어나는 아이들의 이름을 지어달라고 한다.  
부부들은 이름은 잘 짓는게 출세에 영향을 미친다고 생각을 하고 있으며, 따라서 좋은 이름을 지어 출세하기를 기원한다.  
허나 게으른 외수에게 작명은 지루한 작업이다. 효율적으로 일을 하고자 궁리하던 차에 쉽지만 기가 막힌 알고리즘을 고안하게 되었다.  
  
외수가 개발한 작명 알고리즘은 다음과 같다.  
  
아버지의 이름 뒤에 어머니의 이름을 덧붙여서 하나의 새로운 문자열 S로 만든다.  
이 문자열 S의 접두사(prefix)도 되고 접미사(suffix)도 되는 문자열을 찾는다.  
예를 들어 아버지의 이름이 'ala'고 어머니의 이름이 'la' 일 경우 S = 'ala' + 'la' = alala다.  
그리고 이 문자열의 접두사이기도 하고 접미사이기도 한 문자열은 다음 세가지다.  
  
a  
ala  
alala  
  
아버지와 어머니의 이름이 주어질 때, 외수의 규칙을 이용해 지어줄 수 있는 이름들을 모두 찾는 프로그램을 작성하라.  
문제에서는 편의상 모든 문자열 대신, 가능한 모든 문자열의 길이를 찾는다.  
  
#### 입력
빈칸이 없는 영문 알파벳 소문자로 이뤄진 문자열이 입력이 두 줄 입력된다.  
첫 번째 줄은 아버지의 이름이고, 두 번째 줄은 어머니의 이름이다.  
두 문자열의 길이를 합쳐서 400,000 자가 넘어가는 입력은 들어오지 않는다.  
  
#### 출력
외수가 주어질 수 있는 이름들의 길이들을 한 줄에 출력한다.  
출력되는 숫자 사이에는 정확히 공백이 하나 포함되어야 하며, 길이는 오름차순으로 출력되어야 한다.  
  
#### 예제 입력
ababcabababa  
bcabab  
  
#### 예제 출력
2 4 9 18  

### -----------------------------------------------------------------------------------------------------------------

긴 문자열 S가 주어질 때 이 문자열의 접미사도 되고 접두사도 되는 문자열의 길이를 전부 출력하는 문제.
  
KMP 알고리즘의 부분 일치 테이블을 이용하면 S의 접두사도 되고 접미사도 되는 문자열의 최대 길이 k를 알 수 있다.  
이보다 짧은 답들은??  
  
S의 길이 k인 접두사와 접미사는 서로 같으므로 길이가 k 이하인 S의 접미사는 S[...k-1]의 접미사이기도 하다.  
따라서 k 다음으로 긴 접미사를 찾기위해 pi[k-1]을 확인하면 된다. 이 과정을 반복.
    
    // s의 접두사도 되고 접미사도 되는 문자열들의 길이 반환
    vector<int> getPrefixSuffix(const string& s) {
        vector<int> ret, pi = getPartialMatch(s);
        int k = s.size();
        while (k > 0) {
            // s[..k-1]는 답이다.
            ret.push_back(k);
            // s[..k-1]의 접미사도 되고 접두사도 되는 문자열도 답이다.
            k = pi[k-1];
        }
        return ret;
    }

In [7]:
S = input().rstrip() + input().rstrip()

def getPartialMatch(N):
    m = len(N)
    pi = [0] * m
    begin = 1
    matched = 0
    while begin + matched < m:
        if N[begin + matched] == N[matched]:
            matched += 1
            pi[begin + matched - 1] = matched
        else:
            if not matched:
                begin += 1
            else:
                begin += matched - pi[matched - 1]
                matched = pi[matched - 1]
    return pi

def getPrefixSuffix(s):
    ret = []
    pi = getPartialMatch(s)
    k = len(s)
    while k > 0:
        # 자기 자신은 항상 답이 됨.
        ret.append(k)  
        k = pi[k - 1]
    return reversed(ret)

print(*getPrefixSuffix(S))

ababcabababa
bcabab
2 4 9 18


파이썬은 시간 안되네.. 
#### C++ 구현

```c++
#include <iostream>
#include <string>
#include <vector>
#include <stack>
using namespace std;

vector<int> getPartialMatch(const string& N) {
    int m = N.size();
    vector<int> pi(m, 0);
    // KMP로 찾기.
    // N을 N에서 찾는다. begin = 0이면 자기 자신을 찾아버리니까 안됨.
    int begin = 1, matched = 0;
    // 비교할 문자가 N의 끝에 도달할 때까지 찾으면서 부분 일치를 모두 기록.
    while (begin + matched < m) {
        if (N[begin + matched] == N[matched]) {
            ++matched;
            pi[begin + matched - 1] = matched;
        }
        else {
            if (matched == 0)
                ++begin;
            else {
                begin += matched - pi[matched - 1];
                matched = pi[matched - 1];
            }
        }
    }
    return pi;
}

// s의 접두사도 되고 접미사도 되는 문자열들의 길이 반환
stack<int> getPrefixSuffix(const string& s) {
    stack<int> ret;
    vector<int> pi = getPartialMatch(s);
    int k = s.size();
    while (k > 0) {
        // s[..k-1]는 답이다.
        ret.push(k);
        // s[..k-1]의 접미사도 되고 접두사도 되는 문자열도 답이다.
        k = pi[k-1];
    }
    return ret;
}

int main() {
    string a, b;
    cin >> a >> b;
    stack<int> ret = getPrefixSuffix(a + b);
    while(!ret.empty()){
        cout << ret.top() << ' ';
        ret.pop();
    }
    return 0;
}
```

알고스팟 고인물 코드..
```c++
#include <iostream> 
#include <cstring>
#include <cmath>
#include <cstdio>
using namespace std; 
 
char s1[400001];
int table[400001];
int result[400001];
int main()
{
    gets(s1);
    gets(s1+strlen(s1));
 
    int begin = 1, matched = 0, len = strlen(s1);
    while(begin+matched < len)
    {
        if(s1[matched] == s1[begin+matched]){
            matched++;
            table[begin+matched-1] = matched;
        }else{
            if(matched == 0) begin++;
            else{
                begin += matched - table[matched];
                matched = table[matched];
            }
        }
    }
    int ridx = 1;
    result[0] = len;
    while(table[len-1] != 0)
    {
        result[ridx++] = table[len-1];
        len = table[len-1];
    }
 
    for(int i = ridx-1; i >= 0; i--)
    {
        printf("%d%c", result[i], i?' ':'\n');
    }
    return 0;
}
```

#### 팰린드롬 만들기
#### 문제
앞에서부터 읽었을 때와 뒤로부터 읽었을 때 똑같은 문자열을 팰린드롬(palindrome)이라고 합니다.  
예를 들면 “noon”이나 “stats” 같은 단어들이 팰린드롬입니다.  
주어진 문자열 S 뒤에 적절히 문자열을 붙여서 S 를 팰린드롬으로 만들려고 합니다.  
예를 들어 S = “anon”이면 뒤에 “ona”를 붙여서 “anonona”를 만들 수도 있고, “a”를 붙여서 “anona”를 만들 수도 있지요.  
물론 S를 뒤집은 문자열을 S 뒤에 붙이면 항상 팰린드롬이 되므로, 결과 팰린드롬이 가능한 한 짧았으면 합니다.  
  
S가 주어질 때 S에서 만들 수 있는 가장 짧은 팰린드롬의 길이를 출력하는 프로그램을 작성하세요.  
  
#### 입력
입력의 첫 줄에는 테스트 케이스의 수 C(<=50)가 주어집니다.  
그 후 각 테스트 케이스마다 문자열 S가 주어집니다. 주어지는 문자열의 길이는 1 이상 10만 이하이며, 알파벳 소문자로만 구성됩니다.  
  
#### 출력
각 테스트 케이스마다 한 줄에 S를 이용해 만들 수 있는 가장 짧은 팰린드롬의 길이를 출력합니다.  
  
#### 예제 입력
3  
there  
amanaplanacanal  
xyz  
#### 예제 출력
7  
21  
5  

### ---------------------------------------------------------------------------------------------------------------------------------------

간단하게는 우선 S를 뒤집은 문자열 S'을 만든 뒤, S의 접미사이면서 S'의 접두사인 문자열을 찾으면 된다.  
그리고 역시 문자열 검색으로 풀 수 있다.  
  
S와 S'가 주어질 때, S의 접미사이면서 S'의 접두사도 되는 문자열의 최대 길이를 계산하는 maxOverlap()함수 구현..  
시작 위치 제한을 없애고 a의 마지막 글자와 b의 문자가 서로 일치했을 때 지금까지 일치한 부분의 길이를 반환.
```c++
// a의 접미사이면서 b의 접두사인 문자열의 최대 길이를 구한다.
int maxOverlap(const string& a, const string& b) {
    int n = a.size(), m = b.size();
    vector<int> pi = getPartialMatch(b);
    // begin==matched=0에서부터 시작.
    int begin = 0, matched = 0;
    while (begin < n) {
        // 만약 짚더미의 해당 글자가 바늘의 해당 글자와 같다면
        if (matched < m && a[begin + matched] == b[matched]) {
            ++matched;
            if (begin + matched == n)
                return matched;
        }
        else {
            if (matched == 0)
                ++begin;
            else {
                begin += matched - pi[matched - 1];
                matched = pi[matched - 1];
            }
        }
    }
    return 0;
}
```

In [3]:
def getPartialMatch(N):
    m = len(N)
    pi = [0] * m
    begin = 1
    matched = 0
    while begin + matched < m:
        if N[begin + matched] == N[matched]:
            matched += 1
            pi[begin + matched - 1] = matched
        else:
            if not matched:
                begin += 1
            else:
                begin += matched - pi[matched - 1]
                matched = pi[matched - 1]
    return pi

def maxOverlap(a, b):
    n = len(a)
    m = len(b)
    pi = getPartialMatch(b)
    
    begin = matched = 0
    while begin < n:
        if matched < m and a[begin + matched] == b[matched]:
            matched += 1
            if begin + matched == n:
                return matched
        else:
            if matched == 0:
                begin += 1
            else:
                begin += matched - pi[matched - 1]
                matched = pi[matched - 1]
    
    return 0
        
for _ in range(int(input())):
    S = input().rstrip()
    print(2 * len(S) - maxOverlap(S, S[::-1]))

3
there
7
amanaplanacanal
21
xyz
5


어떤 긴 문자열 H와, Pattern 문자열 N이 있을 때, N과 일치하는 H의 부분 문자열의 수 구하는 문제는 어떨까?    
붙어있는 문자열 말고 건너뛰면서도 세는 것.. 
예를들어 H = "ababccdffga", N = "abg"라면,  
H에서 0, 1, 9번째를 뽑아 "abg"를 만들 수 있고, 또 (0, 3, 9), (3, 4, 9)를 뽑아 만들 수 있게.. 

### KMP 알고리즘의 다른 구현.  
전통적인 구현은 앞서 구현방법과 약간 다르다. 이해하기 까다롭지만 더욱 간결.  
지금까지 대응된 문자의 수 matched만을 유지하면서 모든 글자를 순회하고, 각 글자마다 matched를 적절하게 갱신.  
```c++
// KMP알고리즘 다른 구현
vector<int> kmpSearch2(const string& H, const string& N) {
    int n = H.size(), m = N.size();
    vector<int> ret;
    vector<int> pi = getPartialMatch(N);
    // 현재 대응된 글자의 수
    int matched = 0;
    // 각 글자 순회
    for (int i = 0; i < n; i++) {
        // matched번 글자와 H의 해당 글자가 불일치할 경우,
        // 현재 대응된 글자의 수를 pi[matched - 1]로 줄인다.
        while (matched > 0 && H[i] != N[matched])
            matched = pi[matched - 1];
        // 글자가 대응될 경우
        if (H[i] == N[matched]) {
            ++matched;
            if (matched == m) {
                ret.push_back(i - m + 1);
                matched = pi[matched - 1];
            }
        }
    }
    return ret;
}
```

### 재하의 금고(JAEHASAFE)
일곱 살 재하는 이번에 소중한 물건들을 보관하기 위해 어린이용 금고를 샀습니다.  
재하의 취향에 맞춰 다이얼의 둘레에는 숫자가 아니라 여러 가지 동물들이 그려져 있습니다.  
금고를 열기 위해서는 다이얼을 방향을 번갈아 가며 정해진 위치까지 돌려야 합니다.  
위 그림의 예에서 세 다이얼은 각각 다이얼의 현재 상태, 시계 방향으로 돌려서 나와야 하는 상태, 시계 반대 방향으로 나와야 하는 상태를 가리킵니다.  
따라서 시계 방향으로 네 칸, 시계 반대 방향으로 여섯 칸을 돌리면 이 금고를 열 수 있습니다.  
재하는 아빠를 닮아서 성격이 급합니다. 다이얼의 현재 상태와 금고를 여는 방법이 주어졌을 때, 금고를 열기 위해서는 다이얼을 최소 몇 칸 돌려야 할까요?  
  
#### 입력
첫 줄 테스트 케이스 C(1<=C<=50).  
각 테스트 케이스 첫 줄에는 금고를 열기 위해 맞춰야 하는 상태의 수 N(1<=N<=100).
그 다음 N + 1줄에 각 하나씩 금고 다이얼의 상태가 주어짐.  
다이얼의 상태는 맨 위부터 시계방향으로 각 칸에 어떤 그림이 그려져 있는지로 주어짐.  
각 그림은 알파벳 소문자 혹은 대문자로 표현되며, 각 상태는 다이얼의 칸 수를 길이로 하는 문자열임.  10000 이하의 길이.  
주어진 첫 번째 상태는 현재 다이얼의 상태.  
여기서 시계 방향으로 돌려 두 번째 상태를 만들고, 그 다음엔 방향을 반대로 다음 상태를 만드는 일을 반복..  
마지막 상태에 도달하면 금고가 열림. 연속으로 같은 상태가 주어지는 일은 없다.
  
#### 출력
테스트 케이스마다 다이얼을 최소 몇 칸 돌려야 금고를 열 수 있는지 출력
  
#### 예제 입력
```
2
3
abbab
babab
ababb
bbaba
2
RMDCMRCD
MRCDRMDC
DCMRCDRM
```
  
#### 예제 출력
```
6
10
```

### 재하의 금고 풀이

주어진 문자열을 몇 칸이나 환형 시프트(Circular shift)해야 다른 문자열을 얻을 수 있는지 찾는 문제.  
환형 시프트란 문자열의 첫 글자를 맨 뒤로 옮기고 다른 글자들을 모두 한 칸씩 앞으로 옮기는 연산.  
다음과 같은 함수를 구현하여 풀 수 있다.  
shifts(original, target) = 문자열 original을 target으로 만들기 위해 환형 시프트를 몇 번이나 해야 하는지 반환  

#### 환형 시프트 연산
shifts()를 구현하는 가장 간단한 방법은 original에 환형 시프트를 한 번씩 적용하여 target과 비교하는 것.  
그러나 두 문자열을 비교하는 데 최악의 경우 두 문자열의 길이에 비례하는 시간.. 길이 L에 대해 $O(L^2)$이 걸림.  
더 빠른 shifts()를 구현하려면? 두 가지 방법 가능.  
1. 한 테스트 케이스 안에서 다이얼을 나타내는 문자열은 변하지 않는다는 점을 이용해 단순한 접근을 최적화하는 것.  
    - 각 테스트 케이스를 풀기 시작할 때 한 번 다이얼 문자열을 전처리해서, 일일이 각 시프트를 시도해 볼 필요 없이 어느 글자가 어느 위치에 있는지를 색인.
2. 검색 문제로 바꿔서 푸는 것. 좀 더 간단.
    - 원 문자열 original을 s번 시프트했을 때 target을 얻을 수 있다고 가정하자.
    - 이때 original에서 target을 검색하면 시작 위치 s에서 두 문자열의 남은 부분이 전부 일치하게 된다.
    - original을 두 번 이어붙인 긴 문자열 $original^2$에서 target을 찾으면 검색에서 얻은 결과 위치가 우리가 필요한 시프트 개수이다.
```c++
// shifts() 구현
int shifts(const string& original, const string& target) {
    return kmpSearch(original + original, target)[0];
}
```
3. 시계 방향으로 돌리기
    - 매개변수의 순서만 바꿔서 호출하면 된다!
    - a를 시계방향으로 돌려 b를 만드는 데 필요한 칸의 수는 b를 반시계방향으로 돌려 a로 만드는 데 필요한 칸의 수와 같다.

## 접미사 배열(Suffix array)
'문자열의 맥가이버 칼'이라고도 불리며 굉장히 다양한 문자열 문제를 푸는 데 사용된다.  
**어떤 문자열 S의 모든 접미사를 사전순으로 정렬해 둔 것!**  
이를 모두 저장하려면 문자열 길이 제곱에 비례하는 메모리가 필요하기 때문에,  
대개 접미사 배열은 각 접미사의 시작 위치를 담는 정수 배열로 구현된다.  

```
문자열 "alohomora"의 접미사 배열 A[]와 각 위치에서 시작하는 접미사들의 표

---------------------------------------------------------------------------------
i      A[i]              S[A[i]...]
---------------------------------------------------------------------------------
0      8                  a
1      0                  alohomora
2      3                  homora
3      1                  lohomora
4      5                  mora
5      2                  ohomora
6      4                  omora
7      6                  ora
8      7                  ra
---------------------------------------------------------------------------------
```

#### 접미사 배열을 이용한 검색
긴 문자열 H가 N을 포함한다면 항상 N은 H의 어떤 접미사의 접두사라는 점을 이용.  
"alohomora"에서 "homo"를 찾는다면, "homo"는 "alohomora"의 접미사인 "homora"의 접두사가 된다.  
모든 부분 문자열에 대해 이 속성이 성립함.  
H의 접미사 배열을 이진 탐색해서 각 문자열이 출현하는 위치를 찾을 수 있다.  
접미사 배열의 길이는 항상 |H|이므로 이진 탐색의 내부는 $O(log|H|)$번 수행.  
각 문자열 비교에 O(|N|)시간이 걸리기 때문에 이 이진 탐색 수행시간은 $O(|N|log|H|)$.  
전처리 과정에서 H의 접미사 배열을 생성해야 하는 부담이 있지만, 같은 문자열에서 여러 패턴을 찾아야 할 때 매우 유용.

#### 접미사 배열의 생성
일반적 정렬 알고리즘을 사용하면 가장 간단하게 접미사 배열을 만들 수 있다.  
문자열의 길이가 n일 때, [0, n-1]범위의 정수를 모두 담은 정수 배열을 정렬하되, 두 정수를 비교할 때 해당 위치에서 시작하는 접미사들을 비교.  
별도의 비교자를 구현해서 C++ STL의 sort() 함수로 접미사 배열을 만드는 함수의 구현.
```c++
// 두 접미사의 시작 위치 i, j가 주어질 때 두 접미사 중 어느 쪽이 앞에 와야 할지 비교한다.
struct SuffixComparator {
    const string& s;
    SuffixComparator(const string& s) : s(s) {}
    bool operator () (int i, int j) {
        // s.substr() 대신 strcmp()를 쓰면 임시 객체를 만드는 비용이 절약됨.
        return strcmp(s.c_str() + i, s.c_str() + j) < 0;
    }
};
// s의 접미사 배열을 계산한다.
vector<int> getSuffixArrayNaive(const string& s) {
    // 접미사 시작 위치를 담은 배열을 만든다.
    vector<int> perm;
    for (int i = 0; i < s.size(); ++i) perm.push_back(i);
    // 접미사를 비교하는 비교자를 이용해 정렬하면 완성!
    sort(perm.begin() perm.end(), SuffixComparator(s));
    return perm;
}
```
두 문자열을 비교하는데 최대 두 문자열의 길이에 비례. sort()는 $O(nlogn)$번 비교. 전체 복잡도는 $O(n^2logn)$  
하지만 실제로는 몇 글자만 비교해도 판단할 수 있는 경우가 많다. "aaaa....aaaaa"이런경우 말고..

### 맨버-마이어스 알고리즘
가장 빠른 접미사 배열 생성 알고리즘은 O(N)시간에 동작. 하지만 너무 복잡하다.  
맨버-마이어스 알고리즘은 적절히 빠르면서 구현이 간편.  
우리가 정렬하는 문자열들이 한 문자열의 접미사라는 점을 이용하여 수행 시간은 낮춘다.  
접미사들의 목록을 여러 번 정렬하는데, 매번 그 기준을 바꾼다.  
처음에는 접미사의 첫 한 글자만을 기준으로 정렬하고, 다음에는 첫 두 글자를 기준으로 정렬하고,  
그 다음에는 첫 네 글자를 기준으로 정렬.  
이렇게 log n번 정렬을 하고 나면 우리가 원하는 접미사 배열을 얻게 된다.  
정렬을 여러 번 하는데도 더 빠르게 동작하는 이유는 이전 정렬에서 얻은 정보를 이용해 두 문자열의 대소 비교를 O(1)에 할 수 있기 때문.  
  
첫 글자를 기준으로 정렬한 결과를 가지고 있을 때, 첫 글자가 같은 것들끼리 그룹으로 묶고, 각 그룹마다 0부터 시작하는 번호를 매긴다.  
group[i] = S[i...]가 속한 그룹의 번호인 배열 group[]을 만든다.  
  
접미사를 맨 앞에서부터 순회하면서 각 접미사에 그룹 번호를 부여.  
첫 t글자를 기준으로 만든 group[]이 있으면 두 접미사 S[i..], S[j...]중 첫 t글자를 기준으로 어느 쪽이 더 앞에 오는지 쉽게 알 수 있다.  
group[i]와 group[j]중 어느 쪽이 더 작은지만 확인하면 된다.  
또 이것만 있으면 S[i...]와 S[j...]중 첫 2t글자를 기준으로 어느 쪽이 사전에서 앞에 오는지도 상수 시간에 판단할 수 있다.  
S[i...]와 S[j...]가 주어질 때, 우선 첫 t글자를 비교. 만약 이들이 다르면 나머지는 볼 필요도 없다.  
만약 두 접미사가 같은 그룹에 속한다면 S[i+t...]와 S[j+t...]의 첫 t글자를 비교하면 된다.  
  
```c++
// 각 접미사들의 첫 t글자를 기준으로 한 그룹 번호가 주어질 때,
// 주어진 두 접미사를 첫 2t글자를 기준으로 비교한다.
// group[]은 길이가 0인 접미사도 포함한다.
struct Comparator {
    const vector<int>& group;
    int t;
    Comparator(const vector<int>& _group, int _t): group(_group), t(_t) {
        group = _group; t = _t;
    }
    bool operator () (int a, int b) {
        // 첫 t글자가 다르면 이들을 이용해 비교
        if (group[a] != group[b]) return group[a] < group[b];
        // 아니라면 S[a+t..]와 S[b+t..]의 첫 t글자를 비교.
        return group[a + t] < group[b + t];
    }
};
```
group[a + t]와 group[b + t]의 범위 확인을 안하는 이유?  
이 두값을 참조하는 경우는 두 접미사의 첫 t글자가 같을 때 뿐. 그러기 위해서 두 접미사의 길이는 모두 t이상이어야 한다.  
따라서 이 경우 a + t와 b + t는 최대 n. 따라서 group[n]을 아주 작은 값으로 두면 범위 확인 없이 모든 경우를 처리.  
** 전체 알고리즘 구현**
```c++
// s의 접미사 배열 계산
vector<int> getSuffixArray(const string& s) {
    int n = s.size();
    int t = 1;
    vector<int> group(n + 1);
    for (int i = 0; i < n; i++) group[i] = s[i];
    group[n] = -1;
    // 결과적으로 접미사 배열이 될 반환 값. 이 배열을 lg(n)번 정렬.
    vector<int> perm(n);
    for (int i = 0; i < n; i++) perm[i] = i;
    while (t < n) {
        // group[]은 첫 t글자를 기준으로 계산해 뒀다.
        // 첫 2t글자를 기준으로 perm을 다시 정렬
        Comparator compareUsing2T(group, t);
        sort(perm.begin(), perm.end(), compareUsing2T);
        // 2t글자가 n을 넘는다면 이제 접미사 배열 완성
        t *= 2;
        if (t >= n) break;
        // 2t글자를 기준으로 한 그룹 정보를 만든다.
        vector<int> newGroup(n + 1);
        newGroup[n] = -1;
        newGroup[perm[0]] = 0;
        for (int i = 1; i < n; i++)
            if (compareUsing2T(perm[i - 1], perm[i]))
                newGroup[perm[i]] = newGroup[perm[i - 1]] + 1;
            else
                newGroup[perm[i]] = newGroup[perm[i - 1]];
        group = newGroup;
    }
    return perm;
}
```
t글자를 기준으로 각 접미사를 그룹에 배정한 뒤 이 정보를 이용해 2t글자를 기준으로 정렬하는 과정을 반복.  
- t = 1일 때 실제로 직접 정렬을 하는 것이 아니라 해당 접미사의 첫 글자를 그룹 번호로 배정. 그룹 번호가 0부터 시작하는 정수가 아니게 되지만, 어차피 Comparator는 그룹 번호의 대소관계만을 이용하므로 상관없다.
- group[]의 크기는 n + 1이며, 길이 0인 접미사를 나타내는 group[n]의 값은 항상 -1이다. 길이가 0인 접미사는 접미사 배열의 반환 값에 포함되지는 않지만, Comparator가 group[n]에 접근하기 때문에 필요하다. -1로 고정해두면 된다.
  
while문 내부의 시간 복잡도는 O(nlgn)이 걸리는 정렬이 지배. 전체 시간복잡도는 $O(nlog^2n)$이 된다.

#### 예제: 원형 문자열
길이 n(n<=40000)인 문자열을 끝과 끝이 연결된 원형으로 써보자.  
이 문자열은 항상 시계 방향으로 읽는데, 시작 위치가 어디인지는 정해져 있지 않으므로 n가지 방법으로 읽을 수 있다.  
이 중 사전순으로 가장 앞에 오는 문자열은 뭘까?  
  
#### 풀이
가장 단순한 방법은 n개의 후보를 모두 만들어 비교해서 찾는 것. n-1번의 비교, 각 비교에는 O(n)시간. 전체 $O(N^2)$.  
접미사 배열을 이용해 풀 수 있을까?  문자열이 원형으로 이어져 있어서 어려워 보이지만 트릭 존재.  
S를 두 번 반복한 문자열 $S^2$을 만드는 것!  
이때 S를 읽을 수 있는 방법은 모두 $S^2$의 부분 문자열이 된다.  
따라서 $S^2$의 접미사 배열을 우선 만든 뒤 길이가 n이상인 접미사 중 가장 앞에 오는 것을 찾아내면 된다.  
그러면 접미사 배열을 만드는 시간인 $O(nlog^2n)$시간에 구할 수 있다.
```c++
// 접미사 배열을 사용해 원형 문자열 문제를 해결하는 알고리즘 구현
// 사전순으로 가장 앞에 오는 s의 회전 결과를 구한다.
string minShift(const string& s) {
    string s2 = s + s;
    vector<int> a = getSuffixArray(s2);
    for (int i = 0; i < a.size(); ++i)
        if (a[i] <= s.size())
            return s2.substr(a[i], s.size());
    // 여기로 올 일은 없어야 한다.
    return "__oops__";
}
```

#### 예제: 서로 다른 부분 문자열의 수
길이 n(n <= 4000)인 문자열은 최대 n(n+1)/2개의 부분 문자열을 가질 수 있다. 하지만 이들이 서로 모두 다른 것은 아니다.  
문자열이 주어질 때 서로 다른 부분 문자열의 개수를 세는 문제.
  
#### 풀이
가장 간단한 방법은 각 부분 문자열을 직접 만들어 보면서 이들을 집합 자료 구조에 넣는 것.  
$O(n^2)$의 부분 문자열이 있고 C++의 set<> 자료 구조에 넣으려면 각 문자열마다 O(logn)의 비교를 해야함.  
각 문자열 비교에 O(n)이 걸리므로 전체 시간 복잡도는 $O(n^3logn)$   
해시를 이용해서 필요한 비교 회수를 O(1)으로 줄이면 $O(N^3)$이 되지만 그래도 부족..  
  
접미사 배열로 풀 수 있을까? 우선 주어진 문자열의 접미사 배열을 만든다.  
이때 S의 모든 부분 문자열들은 이 접미사들의 접두사로 표현할 수 있다.  
길이 m인 접미사에는 m개의 접두사가 있을 테니, 부분 문자열 중 중복이 없다면 각 접미사의 길이를 모두 더해서 부분 문자열의 수를 얻을 수 있다.  
중복 문자열들을 제외하려면?  한 부분 문자열이 두 번 이상 출현할 경우 이를 접두사로 갖는 접미사들은 접미사 배열 상에서 항상 인접해 있다!  
따라서 한 부분 문자열이 이미 출현했는지를 알기 위해서는 바로 위 접미사만을 보면 된다.  
각 접미사에 대해 배열에서 자기 앞에 오는 배여로가의 최장 공통 접두사의 길이를 구하고 나면 이만큼은 중복된다는 것을 이용하여 구현.  
commonPrefix()의 수행시간은 O(n)이므로 countSubstrings()의 전체 수행 시간은 $O(n^2)$이 된다.
```c++
// s[i..]와 s[j..]의 공통 접두사의 최대 길이를 계산한다.
int commonPrefix(const string& s, int i, int j) {
    int k = 0;
    while (i < s.size() && j < s.size() && s[i] == s[j]) {
        ++i; ++j; ++k;
    }
    return k;
}
// s의 서로 다른 부분 문자열의 수를 센다.
int countSubstrings(const string& s) {
    vector<int> a = getSuffixArray(s);
    int ret = 0;
    int n = s.size();
    for (int i = 0; i < a.size(); ++i) {
        int cp = 0;
        if (i > 0) cp = commonPrefix(s, a[i - 1], a[i]);
        // a[i]의 (n-a[i])개의 접두사들 중에서 cp개는 중복이다.
        ret += n - a[i] - cp;
    }
    return ret;
}
```

#### 문제: 말버릇(HABIT)
정박사가 지금까지 했던 발표들과 강의들에서 했던 말을 모두 대본으로 만들었을 때 K번 이상 등장하는 부분 문자열을 '말버릇'이라고 하자.  
가장 긴 말버릇의 길이를 찾아보자.
  
#### 입력
첫 줄에 테스트 케이스의 수 C(1<=C<=50)  
각 테스트 케이스의 첫 줄에는 K(1<=K<=1000), 두 번째 줄에는 발표 대본이 주어짐.  
대본은 공백 없이 알파벳 소문자로만 구성. 길이는 4000이하.  
  
#### 출력
K번 이상 등장하는 부분 문자열의 최대 길이를 출력.  
어떤 부분 문자열도 K번 이상 등장하지 안는다면 0을 출력.
  
#### 예제 입력
```
4
2
uhmhellouhmmynameislibe
3
banana
1
thatsagoodquestion
3
hello
```
#### 예제 출력
```
3
1
18
0
```

#### 풀이
이 문제 역시 접미사 배열을 이용하여 해결.  
부분 문자열이 여러 번 출현할 경우 항상 인접한 접미사들의 접두사로 출현한다는 사실 이용.  
따라서 두 번 이상 출현하는 부분 문자열의 최대 길이를 알고 싶다면, 배열에서 인접한 모든 접미사 쌍에 대해 최장 공통 접두사를 계산.  
이를 확장시켜 K번 이상 출현하는 부분 문자열을 찾으려면?  
  
어떤 부분 문자열 X가 출현하는 위치가 K군데 있다고 하자. 그러면 X는 k개의 접미사의 접두사가 된다.  
이들 접미사는 항상 접미사 배열에서 인접해 있음. 이 중 첫 번째 접미사와 마지막 접미사의 공통 접두사는 항상 X를 포함한다.  
  
"banana"에서 a는 시작위치 1, 3, 5 세 군데에서 출현.  
따라서 접미사 배열에서 S[1...], S[3...], S[5...]는 항상 인접해 있을 것.  
이 중 첫 접미사와 마지막 접미사의 최장 공통 접두사는 "a"가 된다.  
따라서 모든 A[i]와 A[i + k - 1]의 쌍에 대해 해당하는 두 접미사의 최장 공통 접두사를 계산하는 것으로 이 문제를 풀 수 있다.  
commonPrefix()를 O(n)번 호출하기 때문에 전체 시간 복잡도는 $O(n^2)$이 된다.  
```c++
// s[i..]와 s[j..]의 공통 접두사의 최대 길이를 계산한다.
int commonPrefix(const string& s, int i, int j) {
    int k = 0;
    while (i < s.size() && j < s.size() && s[i] == s[j]) {
        ++i; ++j; ++k;
    }
    return k;
}
// k번 이상 출현하는 s의 부분 문자열 중 최대 길이를 찾는다.
int longestFrequent(int k, const string& s) {
    vector<int> a = getSuffixArray(s);
    int ret = 0;
    for (int i = 0; i + k < = s.size(); i++)
        ret = max(ret, commonPrefix(s, a[i], a[i + k - 1]));
    return ret;
}
```

#### 더 읽을거리
- KMP알고리즘에서 정의하는 부분 일치 테이블은 다른 문제에도 유용하게 쓰임.  
그러나 단순히 문자열 검색만을 할 거라면 **보이어-무어(Boyer-Moore)알고리즘**이 더 효율적인 경우가 많다.  
  
- 접미사 배열과 비슷하지만 더 빠른 검색을 가능하게 하는 자료 구조로 접미사 트리(suffix tree)가 있다.  
메모리 사용량이 크기 때문에 제한적인 경우에만 사용할 수 있지만, 더 직관적이고 빠른 검색을 가능하게 한다.  
  
- 접미사 배열 생성 알고리즘은 O(nlogn)에 수행되도록 최적화할 수 있다. sort()를 이용해 O(nlogn)정렬을 하는 대신 일종의 버킷 정렬(bucket sort)을 수행하면 된다.  맨버-마이어스 논문 참조